# RecDP LLM - Dataset Score Assessment

This notebook shows how to use several tools to evaluate the quality score, diversity, toxicity, perplexity and rouge of a dataset.

# Get Started

## 1. Install pyrecdp and dependencies

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -qq -y openjdk-8-jre
! pip install -q pyrecdp --pre
# ! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

## 2. Prepare your data

In [ ]:
%mkdir -p /content/test_data
%cd /content/test_data
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/alpaca_sample_10.parquet

## 3. Score Assement

In [2]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/opt/intel/oneapi/intelpython/latest/envs/llm_data/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


### 3.1 Process with toxicity and perplexity scorer

In [5]:
pipeline = ResumableTextPipeline()
pipeline.enable_statistics()
ops = [
    ParquetReader("/content/test_data/"),
    TextPerplexityScore(),
    TextToxicity(huggingface_config_path="/root/.cache/huggingface/hub/models--xlm-roberta-base"),
    ParquetWriter("ResumableTextPipeline_output-1")
]
pipeline.add_operations(ops)
ret = pipeline.execute()
del pipeline

2023-11-13 07:03:27.156 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:215 - Enabling this option will result in a decrease in execution speed
2023-11-13 07:03:27.160 | INFO     | pyrecdp.core.model_utils:prepare_sentencepiece_model:107 - Loading sentencepiece model...
2023-11-13 07:03:27.238 | INFO     | pyrecdp.core.model_utils:prepare_kenlm_model:125 - Loading kenlm language model...
[DatasetReader, PerfileSourcedParquetReader, TextPerplexityScore, TextToxicity, PerfileParquetWriter]
init ray with total mem of 324137904537


2023-11-13 07:03:31,837	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-13 07:03:33,059	INFO worker.py:1642 -- Started a local Ray instance.


(pid=59708) Parquet Files Sample 0:   0%|          | 0/1 [00:00<?, ?it/s]

(_get_reader pid=59708) /opt/intel/oneapi/intelpython/latest/envs/llm_data/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:259: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_reader pid=59708)   pq_ds.pieces, **prefetch_remote_args
(_get_reader pid=59708) /opt/intel/oneapi/intelpython/latest/envs/llm_data/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:269: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_reader pid=59708)   self._pq_pieces = [_SerializedPiece(p) for p in pq_ds.pieces]
(_get_reader pid=59708) /opt/intel/oneapi/intelpython/latest/envs/llm_data/lib/python3.9/site-packages/ray/data/datasource/parquet_datasource.py:270: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyar

Running 0:   0%|          | 0/36864 [00:00<?, ?it/s]

...

Shuffle Map 2:   0%|          | 0/36864 [00:00<?, ?it/s]

Shuffle Reduce 3:   0%|          | 0/36864 [00:00<?, ?it/s]

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

ResumableTextPipeline, current on alpaca_sample_10.parquet:   0%|          | 0/1 [00:15<?, ?it/s]


KeyboardInterrupt: 

### 3.2 Process with QualityScorer, Diversity and Rouge scorer

In [16]:
pipeline = ResumableTextPipeline()
pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output-2"
ops = [
    ParquetReader("/content/test_data/alpaca_sample_10.parquet"),
    TextDiversityIndicate(out_dir=out_dir, language="en", first_sent=False),
    TextQualityScorer(text_key="text", model="gpt3"),
    RougeScoreDedup(max_ratio=0.7, batch_size=10,score_store_path=os.path.join(out_dir,'RougeScorefiltered.parquet')),
    ParquetWriter(out_dir)
]
pipeline.add_operations(ops)
ret = pipeline.execute()
del pipeline

2023-11-13 07:18:08.316 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:215 - Enabling this option will result in a decrease in execution speed
[DatasetReader, PerfileSourcedParquetReader, TextDiversityIndicate, TextQualityScorer, RougeScoreDedup, PerfileParquetWriter]
Will assign 48 cores and 412162 M memory for spark
per core memory size is 8.385 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark for global tasks started ...
DatasetReader
2023-11-13 07:18:08.664 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:281 - DatasetReader: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
execute with spark for global tasks took 0.003243283019401133 sec
PerfileSourcedParquetReader


ResumableTextPipeline, current on alpaca_sample_10.parquet:   0%|          | 0/1 [00:00<?, ?it/s]

alpaca_sample_10.parquet
TextDiversityIndicate
statistics_decorator spark


TextQualityScorer
statistics_decorator spark
model_name is gpt3
2023-11-13 07:18:18.588 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:prepare_model:122 - Preparing scorer model in [/root/.cache/recdp/models/gpt3_quality_model]...
real_model_path is /root/.cache/recdp/models/gpt3_quality_model


2023-11-13 07:18:20.270 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:predict:252 - Start scoring dataset...
RougeScoreDedup
statistics_decorator spark


...


0it [00:00, ?it/s]

generate_connected_components => duplicates took 0.017523382004583254 sec


2023-11-13 07:18:27.515 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:277 - TextDiversityIndicate: A total of 10 rows of data were processed, using 9.549101114273071 seconds, Get max diversity types 10, Get average diversity types 1.9,Get the std of diversity types 2.8460498941515415
2023-11-13 07:18:27.519 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:277 - TextQualityScorer: A total of 10 rows of data were processed, using 2.182723045349121 seconds, Get average quality score 0.9541670706928078
2023-11-13 07:18:27.527 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:277 - RougeScoreDedup: A total of 10 rows of data were processed, using 6.3509862422943115 seconds, A duplication list containing 0 found, around 0.0% of total data, Sampled, duplication preview: Empty DataFrame
Columns: [id_1, id_2, id_pair, similarity_left, similarity_right, score]
Index: []
2023-11-13 07:18:27.534 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:281 - PerfileParquetWriter: A total of 0 rows of dat

ResumableTextPipeline, current on alpaca_sample_10.parquet: 100%|██████████| 1/1 [00:18<00:00, 18.68s/it]

2023-11-13 07:18:27.539 | INFO     | pyrecdp.LLM.TextPipeline:execute:421 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output-2 for verification.


### 3.3 View score 

In [14]:
import json
# ppl_score = json.load(open("ResumableTextPipeline_output-2/TextPerplexityScore-statistics", "r"))
# toxicity_score = json.load(open("ResumableTextPipeline_output-2/TextToxicity-statistics", "r"))
quality_score = json.load(open("ResumableTextPipeline_output-2/TextQualityScorer-statistics", "r"))
diversity_score = json.load(open("ResumableTextPipeline_output-2/TextDiversityIndicate-statistics", "r"))
rouge_score = json.load(open("ResumableTextPipeline_output-2/RougeScoreDedup-statistics", "r"))

print("Perplexity scores: ", ppl_score)
print("Toxicity scores: ", toxicity_score)
print("Quality scores: ", quality_score)
print("Diversity scores: ", diversity_score)
print("Rouge scores: ", rouge_score)

Quality scores:  {'mean': 0.9541670706928078}
Diversity scores:  {'max': nan, 'mean': nan, 'std': nan}
Rouge scores:  {'dup_num': 0, 'dup_ratio': 0.0}
